In [ ]:
pip install --upgrade tensorflow==2.1.0

In [ ]:
pip install --upgrade keras==2.3.1

In [ ]:
pip list

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls 'drive/My Drive'

'Colab Notebooks'   datasets   generate_imgs   saved_model


In [4]:
import numpy as np
import glob
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import keras as K
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam


Using TensorFlow backend.


In [30]:
def build_generator(z_dim):

    model = Sequential()

    # 8x8x256
    model.add(Dense(8 * 8 * 256, input_dim=z_dim))
    model.add(Reshape((8, 8, 256)))

    # 8x8x256 => 8x8x128
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    # 8x8x128 => 16x16x64
    model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    # 16x16x64 => 32x32x32
    model.add(Conv2DTranspose(32, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    # 32x32x32 => 64x64x16
    model.add(Conv2DTranspose(16, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    # 64x64x16 => 64x64x3
    model.add(Conv2DTranspose(3, kernel_size=3, strides=1, padding='same'))

    # tanh関数を適用して出力
    model.add(Activation('tanh'))

    return model

In [43]:
def build_discriminator(img_shape):

    model = Sequential()

    # 64x64x3 => 32x32x32
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    # 32x32x32 => 16x16x64
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    # 16x16x64 => 8x8x128
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))


    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [27]:
def build_gan(generator, discriminator):

    model = Sequential()

    model.add(generator)
    model.add(discriminator)

    return model


In [41]:
def train(iterations, batch_size, sample_interval):

    losses = []
    accuracies = []
    iteration_checkpoints = []

    dataset = np.load('drive/My Drive/datasets/irasutoya_64.npz')

    x_train = np.array(dataset['X_train'])

     # 本物の画像のラベルは全て1にする
    real = np.ones((batch_size, 1))

    # 偽物の画像のラベルは全て0にする
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        #-------------
        # 識別器の学習
        #-------------

        # 本物の画像集合からランダムにバッチを生成する
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]

        # 偽物の画像からなるバッチを生成する
        z = np.random.normal(-1, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(z)

        # 識別器の学習
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real , d_loss_fake)

        #-------------
        # 生成器の学習
        #-------------
        for i in range(2):
            # ノイズベクトルを生成
            z = np.random.normal(-1, 1, (batch_size, z_dim))    
            # 生成器の学習
            g_loss = gan.train_on_batch(z, real)

        print('\rNo, %d' %(iteration+1), end='')

        if (iteration + 1) % sample_interval == 0:

            # あとで可視化するために損失と精度を保存しておく
            losses.append([d_loss, g_loss])
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # 学習結果の出力
            print('[D loss: %f, acc.: %.2f%%] [G loss: %f]' %(d_loss, 100.0*accuracy, g_loss))

            sample_images(generator, iteration)

            generator.save("drive/My Drive/saved_model/irasutoya/1/%d_Noda.h5" % (iteration+1))


In [38]:
def sample_images(generator, iteration, image_grid_rows=4, image_grid_columns=4):

    # ノイズベクトルを生成する
    z = np.random.normal(-1, 1, (image_grid_rows * image_grid_columns, z_dim))

    # ノイズベクトルから画像を生成する
    gen_imgs = generator.predict(z)

    # 出力の画素値を[0, 1]の範囲にスケーリングする
    gen_imgs = 0.5 * gen_imgs + 0.5


    # 画像からなるグリッドを生成する
    fig, axs = plt.subplots(image_grid_rows, image_grid_columns, figsize=(4, 4), sharey=True, sharex=True)
    count = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # 並べた画像を出力
            axs[i, j].imshow(gen_imgs[count, :, :, :])
            axs[i, j].axis('off')
            count += 1
    
    fig.savefig("drive/My Drive/generate_imgs/irasutoya/1/%d.png" % (iteration+1))
    plt.close()

In [ ]:
img_rows = 64
img_cols = 64
channels = 3

iterations = 1000000
batch_size = 64
sample_interval = 1000

img_shape = (img_rows, img_cols, channels)
z_dim = 100

# 識別器
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# 生成器
generator = build_generator(z_dim)

discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

train(iterations, batch_size, sample_interval)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


No, 2

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


No, 1000[D loss: 0.892081, acc.: 30.47%] [G loss: 0.816203]
No, 2000[D loss: 0.835465, acc.: 27.34%] [G loss: 0.692886]
No, 3000[D loss: 0.544286, acc.: 75.00%] [G loss: 0.311851]
No, 4000[D loss: 0.248101, acc.: 96.88%] [G loss: 0.222163]
No, 5000[D loss: 0.644871, acc.: 63.28%] [G loss: 0.727962]
No, 6000[D loss: 0.362400, acc.: 85.16%] [G loss: 1.285770]
No, 7000[D loss: 0.124064, acc.: 97.66%] [G loss: 0.226725]
No, 8000[D loss: 2.680827, acc.: 11.72%] [G loss: 3.918017]
No, 9000[D loss: 0.297635, acc.: 89.84%] [G loss: 0.964192]
No, 10000[D loss: 0.105564, acc.: 98.44%] [G loss: 0.673275]
No, 11000[D loss: 0.036149, acc.: 99.22%] [G loss: 0.364819]
No, 11128